In [1]:
import pandas as pd

# Źródło
- Dane pobrane z https://dane.gov.pl/pl/institution/33,ministerstwo-spraw-zagranicznych
- Wprowadzono minimalne zmiany:
    - ujednolicono nazwy plików
    - ujednolicono nazwy kolumn: użyto: `obywatelstwo`, `suma`, `A`, `B`, `C`
    - zamieniono kolumne pierwszą na `obywatelstwo` dla danych z lat 2021 i 2022 (2020 nie zawiera informacji o obywatelstwie)
    - usunięto arkusz `tmpA857` z pliku `2022_II_połowa_wizy_obywatelstwo.xls`

# Przygotowawnie danych

In [259]:
df = pd.DataFrame()

## 2020

In [260]:
_df = pd.read_excel("./data/2020_wizy_obywatelstwo.xls", skiprows=[0]).fillna(0)[:-1]
_df = _df.drop(columns=["suma"])
_df['rok'] = 2020

### Niestety 2020 nie ma danych o obywatelstwie dlatego ściągam z Wikipedii informacje o kraju dla każdego z miast, które występują w dokumencie

In [162]:
import requests
import time
from lxml import HTML


def get_city_country(name):
    response = requests.get(f"https://pl.wikipedia.org/wiki/{name}")
    print("getting:",f"https://pl.wikipedia.org/wiki/{name}")
    tree = html.fromstring(response.content)
    return tree.xpath('//a[@title="Państwo"]/../../td/p/a/text()')[0]

In [163]:
data = [] 
for name in _df['placówka']:
    name = "_".join([x[0] + x[1:].lower() for x in name.split()])
    try:
        country = get_city_country(name)
    except IndexError:
        country = None
    print(name, country)
    data.append({"city": name, "country": country})
    time.sleep(1)

getting: https://pl.wikipedia.org/wiki/Abu_Zabi
Abu_Zabi Zjednoczone Emiraty Arabskie
getting: https://pl.wikipedia.org/wiki/Abudża
Abudża Nigeria
getting: https://pl.wikipedia.org/wiki/Addis_Abeba
Addis_Abeba Etiopia
getting: https://pl.wikipedia.org/wiki/Algieria
Algieria None
getting: https://pl.wikipedia.org/wiki/Ałmaty
Ałmaty Kazachstan
getting: https://pl.wikipedia.org/wiki/Amman
Amman Jordania
getting: https://pl.wikipedia.org/wiki/Ankara
Ankara Turcja
getting: https://pl.wikipedia.org/wiki/Astana
Astana Kazachstan
getting: https://pl.wikipedia.org/wiki/Ateny
Ateny Grecja
getting: https://pl.wikipedia.org/wiki/Baku
Baku Azerbejdżan
getting: https://pl.wikipedia.org/wiki/Bangkok
Bangkok Tajlandia
getting: https://pl.wikipedia.org/wiki/Bejrut
Bejrut Liban
getting: https://pl.wikipedia.org/wiki/Belgrad
Belgrad Serbia
getting: https://pl.wikipedia.org/wiki/Berlin
Berlin Niemcy
getting: https://pl.wikipedia.org/wiki/Berno
Berno Szwajcaria
getting: https://pl.wikipedia.org/wiki/Bogota

In [241]:
_df_countries = pd.DataFrame(data)

In [243]:
_df_countries.loc[_df_countries['country'].isnull(), "country"] = [
    "Algieria",
    "Tanzania",
    "Irak",
    "ChRL",
    "Kuwejt",
    "Luksemburg",
    "Meksyk",
    "Rosja",
    "Panama",
    "Maroko",
    "Rosja",
    "Singapur",
    "Izrael",
    "Ukraina"
]

_df_countries['city'] = _df_countries['city'].str.replace("_", " ").str.upper()
_df_countries['country'] = _df_countries['country'].str.replace("_", " ").str.upper()
_df_countries.to_csv("./data/miasta_panstwa_mapowanie.csv", index=False)

In [261]:
_df_countries = pd.read_csv("./data/miasta_panstwa_mapowanie.csv")
city_map = _df_countries.set_index("city")["country"].to_dict()

In [262]:
_df['obywatelstwo'] = _df['placówka'].apply(city_map.get).replace({"CHRL": "CHINY"})
_df = _df.drop(columns=['placówka'])

In [263]:
df = pd.concat([df, _df])
_df.head()

,A,C,D,rok,obywatelstwo
0,0.0,481.0,1050,2020,ZJEDNOCZONE EMIRATY ARABSKIE
1,0.0,256.0,980,2020,NIGERIA
2,0.0,112.0,1106,2020,ETIOPIA
3,0.0,547.0,140,2020,ALGIERIA
4,0.0,600.0,1695,2020,KAZACHSTAN


## 2021

In [264]:
_df1 = pd.read_excel("./data/2021_II_połowa_wizy_obywatelstwo.xls", skiprows=[0, 1, 2]).fillna(0)[:-1]
_df2 = pd.read_excel("./data/2021_I_połowa_wizy_obywatelstwo.xls", skiprows=[0, 1, 2]).fillna(0)[:-1]
_df = pd.concat([_df1, _df2]).drop(columns=["suma"])
_df = _df.groupby("obywatelstwo").sum().reset_index()
_df['rok'] = 2021

df = pd.concat([df, _df])
_df.head()

,obywatelstwo,C,D,A,rok
0,AFGANISTAN,42,259.0,0,2021
1,ALBANIA,13,262.0,0,2021
2,ALGIERIA,199,240.0,0,2021
3,ANGOLA,40,60.0,0,2021
4,ANTIGUA I BARBUDA,0,1.0,0,2021


## 2022

In [265]:
_df1 = pd.read_excel("./data/2022_II_połowa_wizy_obywatelstwo.xls", skiprows=[0, 1]).fillna(0)[:-1]
_df2 = pd.read_excel("./data/2022_I_połowa_wizy_obywatelstwo.xls", skiprows=[0, 1]).fillna(0)[:-1]
_df = pd.concat([_df1, _df2]).drop(columns=["suma"])
_df = _df.groupby("obywatelstwo").sum().reset_index()
_df['rok'] = 2022

df = pd.concat([df, _df])
_df.head()

,obywatelstwo,A,C,D,rok
0,AFGANISTAN,0,102,456,2022
1,ALBANIA,0,2,248,2022
2,ALGIERIA,0,1573,351,2022
3,ANDORA,0,1,0,2022
4,ANGOLA,0,296,86,2022


# Zapisanie ostatecznego pliku

In [266]:
df.to_csv("./data/wizy_a_b_c_2022_2022.csv", index=False)